## Create an RDD using filtered_registered_business_sf.csv seperated by commas

In [1]:
from pyspark.sql import SparkSession
ss = SparkSession.builder.getOrCreate()
sc = ss.sparkContext

business = sc.textFile("../Data/SF_business/filtered_registered_business_sf.csv").map(lambda x : x.split(','))
business.first()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/15 11:34:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/15 11:34:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


['94123', 'Tournahu George L', '3301 Broderick St', 'San Francisco', 'CA']

## 1. Using createDataFrame(), create a dataframe
Need to add 

<b><i> from pyspark.sql import SparkSession

ss = SparkSession.builder.getOrCreate()

ss.createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True) </i></b>

In [2]:
business_df = ss.createDataFrame(business)

In [3]:
business_df.show()

+-----+--------------------+--------------------+-------------+---+
|   _1|                  _2|                  _3|           _4| _5|
+-----+--------------------+--------------------+-------------+---+
|94123|   Tournahu George L|   3301 Broderick St|San Francisco| CA|
|94124|Stephens Institut...|    2225 Jerrold Ave|San Francisco| CA|
|94105|Stephens Institut...|180 New Montgomer...|San Francisco| CA|
|94108|Stephens Institut...|       540 Powell St|San Francisco| CA|
|94107|Stephens Institut...|     460 Townsend St|San Francisco| CA|
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco| CA|
|94102|Stephens Institut...|       620 Sutter St|San Francisco| CA|
|94102|Stephens Institut...|       655 Sutter St|San Francisco| CA|
|94109|Stephens Institut...|        1055 Pine St|San Francisco| CA|
|94107|Stephens Institut...|    121 Wisconsin St|San Francisco| CA|
|94102|Stephens Institut...|        150 Hayes St|San Francisco| CA|
|94133|Stephens Institut...|    2300 Stockton St

In [4]:
business_df.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)
 |-- _5: string (nullable = true)



In [ ]:
business_df.show(5)

## 2. Adding schema

In [5]:
from pyspark.sql.types import *
schema = StructType([ StructField("zip", LongType(), True),
                      StructField("name", StringType(), False),
                      StructField("street", StringType(), True),
                      StructField("city", StringType(), True),
                      StructField("state", StringType(), True)
                    ])

In [9]:
business_updated = business.map(lambda x : [int(x[0]), x[1], x[2], x[3], x[4]])

In [10]:
business_df = ss.createDataFrame(business_updated, schema)

### What happens below?

In [14]:
business_df.show(20000)

22/02/15 11:38:32 ERROR Executor: Exception in task 0.0 in stage 7.0 (TID 7)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/dwoodbridge/opt/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/Users/dwoodbridge/opt/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/dwoodbridge/opt/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/dwoodbridge/opt/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 74, in wrapper
    return f(*args, **kwargs)
  File "/var/folders/0q/77jvn_416

Py4JJavaError: An error occurred while calling o129.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7) (10.1.152.3 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/dwoodbridge/opt/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/Users/dwoodbridge/opt/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/dwoodbridge/opt/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/dwoodbridge/opt/anaconda3/envs/Dis

## How to fix it?

In [17]:
def IntegerSafe(value): # In case there are non-integer type to be converted.
    try:
        return int(value)
    except:
        return None

business = business.map(lambda x : (IntegerSafe(x[0]), x[1], x[2], x[3], x[4]))
business_df = ss.createDataFrame(business, schema)
business_df.show(20000)
business_df.printSchema()

+-----+--------------------+--------------------+--------------------+--------------------+
|  zip|                name|              street|                city|               state|
+-----+--------------------+--------------------+--------------------+--------------------+
|94123|   Tournahu George L|   3301 Broderick St|       San Francisco|                  CA|
|94124|Stephens Institut...|    2225 Jerrold Ave|       San Francisco|                  CA|
|94105|Stephens Institut...|180 New Montgomer...|       San Francisco|                  CA|
|94108|Stephens Institut...|       540 Powell St|       San Francisco|                  CA|
|94107|Stephens Institut...|     460 Townsend St|       San Francisco|                  CA|
|94109|Stephens Institut...|1835-49 Van Ness Ave|       San Francisco|                  CA|
|94102|Stephens Institut...|       620 Sutter St|       San Francisco|                  CA|
|94102|Stephens Institut...|       655 Sutter St|       San Francisco|          

In [9]:
ss.stop()